In [1]:
import docker
import uuid
import paramiko
import os
from timeit import default_timer as timer
from dataclasses import dataclass
import re
import requests
import statistics
import pandas as pd
import numpy as np

In [2]:
def run_in_master(command):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect("namenode", username="root", password="pass")
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(f"cd /app/ && . /env_var_path.sh && {command}")
    return (ssh_stdout.readlines(), ssh_stderr.readlines())

def run_in_hive(command):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect("hive-server", username="root", password="pass")
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(f"bash -c '. /env_var_path.sh && {command}'")
    return (ssh_stdout.readlines(), ssh_stderr.readlines())

Create a hive database and schema for the covid table.

In [3]:
run_in_hive("hive -f /data/master_volume/hive_scripts/covid_table.hql")

([],
 ['SLF4J: Class path contains multiple SLF4J bindings.\n',
  'SLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.10.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]\n',
  'SLF4J: Found binding in [jar:file:/opt/hadoop-3.3.1/share/hadoop/common/lib/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]\n',
  'SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.\n',
  'SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]\n',
  'Hive Session ID = 27b0b110-7e3b-4cad-9784-b15deea12537\n',
  '\n',
  'Logging initialized using configuration in file:/opt/hive/conf/hive-log4j2.properties Async: true\n',
  'Hive Session ID = 4e04f172-f382-4433-8cdb-06434c50c539\n',
  'OK\n',
  'Time taken: 0.503 seconds\n',
  'OK\n',
  'Time taken: 0.026 seconds\n',
  'OK\n',
  'Time taken: 0.047 seconds\n',
  'OK\n',
  'Time taken: 0.126 seconds\n'])

Copy the covid-dataset.csv file to the external database folder.

In [4]:
run_in_master(f"hdfs dfs -cp /datasets/covid-dataset.csv /user/hive/warehouse/covid.db/covid/covid-dataset.csv")

([], [])

Process the data and transform into another table.

In [5]:
run_in_hive("hive -f /data/master_volume/hive_scripts/covid_data.hql")

([],
 ['SLF4J: Class path contains multiple SLF4J bindings.\n',
  'SLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.10.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]\n',
  'SLF4J: Found binding in [jar:file:/opt/hadoop-3.3.1/share/hadoop/common/lib/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]\n',
  'SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.\n',
  'SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]\n',
  'Hive Session ID = 96070357-55db-46af-b3ba-1ade9180d78c\n',
  '\n',
  'Logging initialized using configuration in file:/opt/hive/conf/hive-log4j2.properties Async: true\n',
  'Hive Session ID = 0b565f16-3807-45e4-8fa1-6f2904f5d815\n',
  'OK\n',
  'Time taken: 0.419 seconds\n',
  'Query ID = root_20230528085503_548dfd89-19da-4585-b141-6337c9819876\n',
  'Total jobs = 1\n',
  'Launching Job 1 out of 1\n',
  'Number of reduce tasks not specified. Estimated from input data size: 1\n',

Compare the results of the new file with the original.

In [7]:
run_in_master(f"hdfs dfs -cat /user/hive/warehouse/covid.db/covid/covid-dataset.csv")

(['iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hun

In [44]:
run_in_master(f"hdfs dfs -cat /user/hive/warehouse/covid.db/calculated/000000_0")

(['\\N\x01Africa\x01\\N\x01\\N\x01264613\x010\x013\x0110990.92794392809\n',
  '\\N\x01Africa\x01\\N\x01\\N\x01264613\x010\x013\x0110990.92794392809\n',
  '\\N\x01Africa\x01\\N\x01\\N\x01264613\x013\x014\x0110990.92794392809\n',
  '\\N\x01Africa\x01\\N\x01\\N\x01264610\x013\x014\x0110990.92794392809\n',
  '\\N\x01Africa\x01\\N\x01\\N\x01264607\x0110\x014\x0110990.92794392809\n',
  '\\N\x01Africa\x01\\N\x01\\N\x01264597\x014\x013\x0110990.92794392809\n',
  '\\N\x01Africa\x01\\N\x01\\N\x01264593\x012\x013\x0110990.92794392809\n',
  '\\N\x01Africa\x01\\N\x01\\N\x01264591\x015\x013\x0110990.92794392809\n',
  '\\N\x01Africa\x01\\N\x01\\N\x01264586\x012\x013\x0110990.92794392809\n',
  '\\N\x01Africa\x01\\N\x01\\N\x01264584\x012\x014\x0110990.92794392809\n',
  '\\N\x01Africa\x01\\N\x01\\N\x01264582\x014\x014\x0110990.92794392809\n',
  '\\N\x01Africa\x01\\N\x01\\N\x01264578\x012\x014\x0110990.92794392809\n',
  '\\N\x01Africa\x01\\N\x01\\N\x01264576\x015\x014\x0110990.92794392809\n',
  '\\N\x01A